In [21]:
import os

In [22]:
%pwd

'd:\\Text Summarization\\Text-Summarizer'

In [8]:
#os.chdir("../")

In [23]:
%pwd

'd:\\Text Summarization\\Text-Summarizer'

In [24]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file:Path
    unzip_dir:Path


In [25]:
from Text_Summarizer.constants import *
from Text_Summarizer.logging import logger
from Text_Summarizer.utils.common import read_yaml_file, create_directories
import os


class ConfigurationManager:
    def __init__(self, config_file_path = CONFIG_PATH,
                 params_file_path = PARAMS_PATH):
        self.config = read_yaml_file(config_file_path)
        self.params = read_yaml_file(params_file_path)
        
        create_directories([self.config.artifacts_root])
    

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
        root_dir=config.root_dir,
        source_url=config.source_url,
        local_data_file=config.local_data_file,
        unzip_dir=config.unzip_dir,
    
        
    )
        return data_ingestion_config


In [26]:
import os
from Text_Summarizer.constants import *
from Text_Summarizer.logging import logger
import zipfile
import urllib.request as request
import os
from Text_Summarizer.utils.common import get_size


class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config
    
    
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url= self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} dowloaded the following info: \n{headers}")
        else:
            logger.info(f"file already exists of size{get_size(Path(self.config.local_data_file))}")   
            
            
    def unzip_data(self):
        
        unzip_path  = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)


In [27]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.unzip_data()
except Exception as e:
    raise e


[2024-06-28 17:22:16,330] : INFO:  common:  Successfully read yaml file: config\config.yaml] 
[2024-06-28 17:22:16,331] : INFO:  common:  Successfully read yaml file: params.yaml] 
[2024-06-28 17:22:16,334] : INFO:  common:  Successfully created directories: ['artifacts']] 
[2024-06-28 17:22:16,336] : INFO:  common:  Successfully created directories: ['artifacts/data_ingestion']] 
[2024-06-28 17:22:23,429] : INFO:  3756739619:  artifacts/data_ingestion/samsundata.zip dowloaded the following info: 
Connection: close
Content-Length: 4194846
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "1637c4924cab7415e900f4a0fd3d9f47fccc6b12b083dbefef19ca7bcb438bfa"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1793:E13C:23004C:2A8F39:667EA2B7
Accept-Ranges: bytes
Date: Fri, 28 Jun 2024 11:52:16 GMT
Via: